In this project we will build a day tarding, algorithmic trading model, and will develop a trading startegy based on the model.

This model is built based on only Bitcoin and Ethereum. Last two years of data is used to build the model. The assumption is that the model finds the best deals between 1 and 2 PM, and liquidates the position at 9 PM (if the position is not liquidated by then). Take Profit is set at 2% return and Stop Loss is set at -1% return.

At the end of the code, it is explained what possible changes you can make for your reports. Before that, make sure you can follow the code from the beginning to the end.

In [4]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [11]:
data=pd.read_csv("drive/My Drive/BTC_ETH.csv")

Data Exploration

In [6]:
data.shape

(226269, 7)

In [7]:
data.tail(2)

,low,high,open,close,volume,date,pair
226267,1343.50,1344.95,1344.41,1344.53,25.116376,10/25/2022 0:40,ETH-USDT
226268,1343.98,1344.67,1343.98,1344.67,4.118600,10/25/2022 0:35,ETH-USDT


In [8]:
data.dtypes

low       float64
high      float64
open      float64
close     float64
volume    float64
date       object
pair       object
dtype: object

In [12]:
# since date column is Character, we will convert it to date_time object, so we can extract date and time parts, and do other date_time operations
import datetime as dt

data['date_time'] = pd.to_datetime(data['date'])

In [13]:
data['time'] = data['date_time'].apply(lambda x: x.time())
data.head(2)

,low,high,open,close,volume,date,pair,date_time,time
0,42281.98,42358.22,42281.98,42348.03,5.815467,12/15/2023 14:10,BTC-USDT,2023-12-15 14:10:00,14:10:00
1,42261.65,42352.55,42352.55,42286.05,1.591771,12/15/2023 14:05,BTC-USDT,2023-12-15 14:05:00,14:05:00


In [14]:
data['date'] = data['date_time'].apply(lambda x: x.date())
data.head(2)

,low,high,open,close,volume,date,pair,date_time,time
0,42281.98,42358.22,42281.98,42348.03,5.815467,2023-12-15,BTC-USDT,2023-12-15 14:10:00,14:10:00
1,42261.65,42352.55,42352.55,42286.05,1.591771,2023-12-15,BTC-USDT,2023-12-15 14:05:00,14:05:00


In [15]:
# we also define a column that shows date_time as number of seconds from the base point (which is 1/1/1970).
# This column will be useful for our SQL operations later.

data['time_in_seconds'] = data['date_time'].apply(lambda x: x.timestamp())

In [16]:
data.tail(2)

,low,high,open,close,volume,date,pair,date_time,time,time_in_seconds
226267,1343.50,1344.95,1344.41,1344.53,25.116376,2022-10-25,ETH-USDT,2022-10-25 00:40:00,00:40:00,1.666658e+09
226268,1343.98,1344.67,1343.98,1344.67,4.118600,2022-10-25,ETH-USDT,2022-10-25 00:35:00,00:35:00,1.666658e+09


In [17]:
data.dtypes

low                       float64
high                      float64
open                      float64
close                     float64
volume                    float64
date                       object
pair                       object
date_time          datetime64[ns]
time                       object
time_in_seconds           float64
dtype: object

Next we define dependent and independent variables. In this simple model, we define a few features based on the average price in the last 1 to 8 hours. Dependent variable is a binary variable which shows whether take profit happened before 9 PM.

In [19]:
import sqlite3

conn = sqlite3.connect("my_db.db")

data.to_sql("data_in_sql", conn, if_exists="replace",index=False)

226269

An important (and time consuming) part of any data project is Sanity Check. In this step you make sure the data is correct. Following is a sanity check in which we make sure data is unique in Pair/Date_Time. As you know, combination of Pair and Date_Time is the unque identifier in this data and should be unique.

# Note: The only issue I have found with sqlite is that once you create a table, you can not use that table name again, to create a table or modify the same table. That would create issues when you want to edit the table, rerun parts of the code, ... You may want to delete and recreate the table if something like that happens. But also it sometimes does not let you delete the table, and says it is locked. Overall this issue can be frustrating. You may need to create totally new tables, ... A model like this should be developed on cloud.

In [20]:
query = """
SELECT distinct pair, date_time, count(date_time) as count
FROM data_in_sql
group by pair, date_time
"""

unique_check = pd.read_sql_query(query, conn)

unique_check[unique_check["count"] > 1]

# sounds like there are three duplicates, probably these records have been recorded twice on the Coinbase side.
# We keep only one.

,pair,date_time,count
101457,BTC-USDT,2023-10-25 13:15:00,2
158879,ETH-USDT,2023-03-29 05:30:00,2
159169,ETH-USDT,2023-03-30 06:30:00,2


In [21]:
conn.execute(
    """
create table data_in_sql_unique as
SELECT distinct *
FROM data_in_sql
    """)

In [22]:
# Let's check again

query = """
SELECT distinct pair, date_time, count(date_time) as count
FROM data_in_sql_unique
group by pair, date_time
"""

unique_check = pd.read_sql_query(query, conn)

unique_check[unique_check["count"] > 1]

# Sounds good

,pair,date_time,count


# Make sure you clearly understand the rationale behind each where condition in the following queries.

In [23]:
# Moving Average in the last 8 hours based on close price.
# Calculation is done only for time between 1 and 2 PM, which is the modeling sample.
# Think about how to calculate MA based on Close, Open, High, and Low?

query = """
SELECT a.pair, a.date_time, avg(b.close) as MA_8
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and a.time_in_seconds-b.time_in_seconds <= 28800
and a.time_in_seconds > b.time_in_seconds
and a.time > "12:59:59"
and a.time < "14:00:01"

group by a.pair, a.date_time
"""
MA_8 = pd.read_sql_query(query, conn)

In [25]:
# MA in the last 4 hours based on close price

query = """
SELECT a.pair, a.date_time, avg(b.close) as MA_4
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and a.time_in_seconds-b.time_in_seconds <= 14400
and a.time_in_seconds > b.time_in_seconds
and a.time > "12:59:59"
and a.time < "14:00:01"

group by a.pair, a.date_time
"""
MA_4 = pd.read_sql_query(query, conn)

In [26]:
# MA in the last 2 hours based on close price

query = """
SELECT a.pair, a.date_time, avg(b.close) as MA_2
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and a.time_in_seconds-b.time_in_seconds <= 7200
and a.time_in_seconds > b.time_in_seconds
and a.time > "12:59:59"
and a.time < "14:00:01"

group by a.pair, a.date_time
"""
MA_2 = pd.read_sql_query(query, conn)

In [29]:
# MA in the last 1 hour based on close price

query = """
SELECT a.pair, a.date_time, avg(b.close) as MA_1
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and a.time_in_seconds-b.time_in_seconds <= 3600
and a.time_in_seconds > b.time_in_seconds
and a.time > "12:59:59"
and a.time < "14:00:01"

group by a.pair, a.date_time
"""
MA_1 = pd.read_sql_query(query, conn)

In [65]:
# MA in the last 30 minutes based on close price

query = """
SELECT a.pair, a.date_time, avg(b.close) as MA_30_min
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and a.time_in_seconds-b.time_in_seconds <= 1800
and a.time_in_seconds > b.time_in_seconds
and a.time > "12:59:59"
and a.time < "14:00:01"

group by a.pair, a.date_time
"""
MA_30 = pd.read_sql_query(query, conn)

In [66]:
# Check if Take Profit happens before 9 PM
# TP is calculated based on close price. You can also define it based on high price, but high can be outlier
# TP is set at 2% return

query = """
SELECT a.pair, a.date_time, min(b.time_in_seconds) as TP_time
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and (b.time_in_seconds-a.time_in_seconds) <= 28800
and b.time_in_seconds > a.time_in_seconds
and b.time < "21:00:01"
and a.time > "12:59:59"
and a.time < "14:00:01"
and b.close >= (1.02*a.close)

group by a.pair, a.date_time
"""
TP = pd.read_sql_query(query, conn)

In [67]:
# Check if Stop Loss happens before 9 PM
# If TP happens but SL happens before that, it will be a SL
# SL is set at -1% return

query = """
SELECT a.pair, a.date_time, min(b.time_in_seconds) as SL_time
FROM data_in_sql_unique as a, data_in_sql_unique as b
where
a.pair = b.pair
and (b.time_in_seconds-a.time_in_seconds) <= 28800
and b.time_in_seconds > a.time_in_seconds
and b.time < "21:00:01"
and a.time > "12:59:59"
and a.time < "14:00:01"
and b.close <= (0.99*a.close)

group by a.pair, a.date_time
"""
SL = pd.read_sql_query(query, conn)

In [ ]:
# Next we merge everything and add the variables created

query = """
SELECT distinct pair, date_time, close
FROM data_in_sql_unique
where
time > "12:59:59"
and time < "14:00:01"

group by pair, date_time
"""

temp1 = pd.read_sql_query(query, conn)

In [32]:
conn.execute(
    """
create table base as
SELECT distinct pair, date_time, close
FROM data_in_sql_unique
where
time > "12:59:59"
and time < "14:00:01"

group by pair, date_time
    """)

In [43]:
# when data processing, merging, ... always do many checks

base = pd.read_sql_query("SELECT * FROM base", conn)
base.shape

# sounds good. count is the same as base table.

(10406, 3)

In [44]:
base.head(4)

,pair,date_time,close
0,BTC-USDT,2022-10-25 13:00:00,19335.83
1,BTC-USDT,2022-10-25 13:05:00,19348.41
2,BTC-USDT,2022-10-25 13:10:00,19390.18
3,BTC-USDT,2022-10-25 13:15:00,19400.22


In [52]:
#MA_8.to_sql("MA_8", conn, index=False)

conn.execute(
    """
create table t1 as
SELECT a.*, b.MA_8
FROM base as a left join MA_8 as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

In [57]:
# count chack is the first check for accuracy of a merge
t1 = pd.read_sql_query("SELECT * FROM t1", conn)
t1.shape

(10406, 4)

In [54]:
t1.head(10)

,pair,date_time,close,MA_8
0,BTC-USDT,2022-10-25 13:00:00,19335.83,19302.483263
1,BTC-USDT,2022-10-25 13:05:00,19348.41,19302.476947
2,BTC-USDT,2022-10-25 13:10:00,19390.18,19302.807684
3,BTC-USDT,2022-10-25 13:15:00,19400.22,19303.567053
4,BTC-USDT,2022-10-25 13:20:00,19393.21,19304.624316
5,BTC-USDT,2022-10-25 13:25:00,19381.16,19305.471789
6,BTC-USDT,2022-10-25 13:30:00,19426.50,19306.228211
7,BTC-USDT,2022-10-25 13:35:00,19397.00,19307.324000
8,BTC-USDT,2022-10-25 13:40:00,19415.17,19308.069789
9,BTC-USDT,2022-10-25 13:45:00,19515.10,19308.980211


In [55]:
MA_4.to_sql("MA_4", conn, index=False)

conn.execute(
    """
create table t2 as
SELECT a.*, b.MA_4
FROM t1 as a left join MA_4 as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

t2 = pd.read_sql_query("SELECT * FROM t2", conn)

t2.head(10)

,pair,date_time,close,MA_8,MA_4
0,BTC-USDT,2022-10-25 13:00:00,19335.83,19302.483263,19287.277292
1,BTC-USDT,2022-10-25 13:05:00,19348.41,19302.476947,19288.516667
2,BTC-USDT,2022-10-25 13:10:00,19390.18,19302.807684,19289.834792
3,BTC-USDT,2022-10-25 13:15:00,19400.22,19303.567053,19292.297500
4,BTC-USDT,2022-10-25 13:20:00,19393.21,19304.624316,19294.592500
5,BTC-USDT,2022-10-25 13:25:00,19381.16,19305.471789,19297.131250
6,BTC-USDT,2022-10-25 13:30:00,19426.50,19306.228211,19299.289583
7,BTC-USDT,2022-10-25 13:35:00,19397.00,19307.324000,19302.572917
8,BTC-USDT,2022-10-25 13:40:00,19415.17,19308.069789,19305.123750
9,BTC-USDT,2022-10-25 13:45:00,19515.10,19308.980211,19308.186667


In [56]:
t2.shape

(10406, 5)

In [58]:
MA_2.to_sql("MA_2", conn, index=False)

conn.execute(
    """
create table t3 as
SELECT a.*, b.MA_2
FROM t2 as a left join MA_2 as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

t3 = pd.read_sql_query("SELECT * FROM t3", conn)

print (t3.shape)

t3.head(10)

(10406, 6)


,pair,date_time,close,MA_8,MA_4,MA_2
0,BTC-USDT,2022-10-25 13:00:00,19335.83,19302.483263,19287.277292,19297.542917
1,BTC-USDT,2022-10-25 13:05:00,19348.41,19302.476947,19288.516667,19299.646667
2,BTC-USDT,2022-10-25 13:10:00,19390.18,19302.807684,19289.834792,19302.358750
3,BTC-USDT,2022-10-25 13:15:00,19400.22,19303.567053,19292.297500,19306.561250
4,BTC-USDT,2022-10-25 13:20:00,19393.21,19304.624316,19294.592500,19311.478333
5,BTC-USDT,2022-10-25 13:25:00,19381.16,19305.471789,19297.131250,19315.726250
6,BTC-USDT,2022-10-25 13:30:00,19426.50,19306.228211,19299.289583,19318.809167
7,BTC-USDT,2022-10-25 13:35:00,19397.00,19307.324000,19302.572917,19323.455000
8,BTC-USDT,2022-10-25 13:40:00,19415.17,19308.069789,19305.123750,19326.747083
9,BTC-USDT,2022-10-25 13:45:00,19515.10,19308.980211,19308.186667,19331.185000


In [59]:
MA_1.to_sql("MA_1", conn, index=False)

conn.execute(
    """
create table t4 as
SELECT a.*, b.MA_1
FROM t3 as a left join MA_1 as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

t4 = pd.read_sql_query("SELECT * FROM t4", conn)

print (t4.shape)

t4.tail(10)

(10406, 7)


,pair,date_time,close,MA_8,MA_4,MA_2,MA_1
10396,ETH-USDT,2023-12-15 13:15:00,2255.57,2270.603723,2272.690000,2266.530435,2262.129167
10397,ETH-USDT,2023-12-15 13:20:00,2253.63,2270.347979,2272.096304,2265.887391,2261.258333
10398,ETH-USDT,2023-12-15 13:25:00,2250.57,2270.101064,2271.434783,2265.025652,2260.428333
10399,ETH-USDT,2023-12-15 13:30:00,2247.68,2269.824255,2270.795652,2264.423333,2259.410833
10400,ETH-USDT,2023-12-15 13:35:00,2245.28,2269.587447,2270.036739,2263.302500,2257.883333
10401,ETH-USDT,2023-12-15 13:40:00,2245.68,2269.384043,2269.240870,2262.033333,2256.125000
10402,ETH-USDT,2023-12-15 13:45:00,2247.50,2269.149681,2268.461087,2260.816667,2254.432500
10403,ETH-USDT,2023-12-15 13:50:00,2247.25,2268.939255,2267.776957,2259.698333,2252.995833
10404,ETH-USDT,2023-12-15 13:55:00,2248.63,2268.685000,2267.159130,2258.652083,2251.682500
10405,ETH-USDT,2023-12-15 14:00:00,2252.01,2268.503191,2266.513913,2257.634167,2250.666667


In [68]:
MA_30.to_sql("MA_30", conn, index=False)

conn.execute(
    """
create table t5 as
SELECT a.*, b.MA_30_min
FROM t4 as a left join MA_30 as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

t5 = pd.read_sql_query("SELECT * FROM t5", conn)

print (t5.shape)

t5.tail(10)

(10406, 8)


,pair,date_time,close,MA_8,MA_4,MA_2,MA_1,MA_30_min
10396,ETH-USDT,2023-12-15 13:15:00,2255.57,2270.603723,2272.690000,2266.530435,2262.129167,2259.130000
10397,ETH-USDT,2023-12-15 13:20:00,2253.63,2270.347979,2272.096304,2265.887391,2261.258333,2257.601667
10398,ETH-USDT,2023-12-15 13:25:00,2250.57,2270.101064,2271.434783,2265.025652,2260.428333,2256.038333
10399,ETH-USDT,2023-12-15 13:30:00,2247.68,2269.824255,2270.795652,2264.423333,2259.410833,2254.330000
10400,ETH-USDT,2023-12-15 13:35:00,2245.28,2269.587447,2270.036739,2263.302500,2257.883333,2253.905000
10401,ETH-USDT,2023-12-15 13:40:00,2245.68,2269.384043,2269.240870,2262.033333,2256.125000,2251.848333
10402,ETH-USDT,2023-12-15 13:45:00,2247.50,2269.149681,2268.461087,2260.816667,2254.432500,2249.735000
10403,ETH-USDT,2023-12-15 13:50:00,2247.25,2268.939255,2267.776957,2259.698333,2252.995833,2248.390000
10404,ETH-USDT,2023-12-15 13:55:00,2248.63,2268.685000,2267.159130,2258.652083,2251.682500,2247.326667
10405,ETH-USDT,2023-12-15 14:00:00,2252.01,2268.503191,2266.513913,2257.634167,2250.666667,2247.003333


In [71]:
#TP.to_sql("TP", conn, index=False)

conn.execute(
    """
create table t6 as
SELECT a.*, b.TP_time
FROM t5 as a left join TP as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

t6 = pd.read_sql_query("SELECT * FROM t6", conn)

print (t6.shape)

t6.tail(10)

(10406, 9)


,pair,date_time,close,MA_8,MA_4,MA_2,MA_1,MA_30_min,TP_time
10396,ETH-USDT,2023-12-15 13:15:00,2255.57,2270.603723,2272.690000,2266.530435,2262.129167,2259.130000,NaN
10397,ETH-USDT,2023-12-15 13:20:00,2253.63,2270.347979,2272.096304,2265.887391,2261.258333,2257.601667,NaN
10398,ETH-USDT,2023-12-15 13:25:00,2250.57,2270.101064,2271.434783,2265.025652,2260.428333,2256.038333,NaN
10399,ETH-USDT,2023-12-15 13:30:00,2247.68,2269.824255,2270.795652,2264.423333,2259.410833,2254.330000,NaN
10400,ETH-USDT,2023-12-15 13:35:00,2245.28,2269.587447,2270.036739,2263.302500,2257.883333,2253.905000,NaN
10401,ETH-USDT,2023-12-15 13:40:00,2245.68,2269.384043,2269.240870,2262.033333,2256.125000,2251.848333,NaN
10402,ETH-USDT,2023-12-15 13:45:00,2247.50,2269.149681,2268.461087,2260.816667,2254.432500,2249.735000,NaN
10403,ETH-USDT,2023-12-15 13:50:00,2247.25,2268.939255,2267.776957,2259.698333,2252.995833,2248.390000,NaN
10404,ETH-USDT,2023-12-15 13:55:00,2248.63,2268.685000,2267.159130,2258.652083,2251.682500,2247.326667,NaN
10405,ETH-USDT,2023-12-15 14:00:00,2252.01,2268.503191,2266.513913,2257.634167,2250.666667,2247.003333,NaN


In [74]:
t6[t6.TP_time.notnull()].shape[0]

1537

In [75]:
SL.to_sql("SL", conn, index=False)

conn.execute(
    """
create table t7 as
SELECT a.*, b.SL_time
FROM t6 as a left join SL as b
on a.pair = b.pair and a.date_time = b.date_time
    """)

t7 = pd.read_sql_query("SELECT * FROM t7", conn)

print (t7.shape)

t7.tail(10)

(10406, 10)


,pair,date_time,close,MA_8,MA_4,MA_2,MA_1,MA_30_min,TP_time,SL_time
10396,ETH-USDT,2023-12-15 13:15:00,2255.57,2270.603723,2272.690000,2266.530435,2262.129167,2259.130000,NaN,1.702654e+09
10397,ETH-USDT,2023-12-15 13:20:00,2253.63,2270.347979,2272.096304,2265.887391,2261.258333,2257.601667,NaN,1.702654e+09
10398,ETH-USDT,2023-12-15 13:25:00,2250.57,2270.101064,2271.434783,2265.025652,2260.428333,2256.038333,NaN,1.702655e+09
10399,ETH-USDT,2023-12-15 13:30:00,2247.68,2269.824255,2270.795652,2264.423333,2259.410833,2254.330000,NaN,NaN
10400,ETH-USDT,2023-12-15 13:35:00,2245.28,2269.587447,2270.036739,2263.302500,2257.883333,2253.905000,NaN,NaN
10401,ETH-USDT,2023-12-15 13:40:00,2245.68,2269.384043,2269.240870,2262.033333,2256.125000,2251.848333,NaN,NaN
10402,ETH-USDT,2023-12-15 13:45:00,2247.50,2269.149681,2268.461087,2260.816667,2254.432500,2249.735000,NaN,NaN
10403,ETH-USDT,2023-12-15 13:50:00,2247.25,2268.939255,2267.776957,2259.698333,2252.995833,2248.390000,NaN,NaN
10404,ETH-USDT,2023-12-15 13:55:00,2248.63,2268.685000,2267.159130,2258.652083,2251.682500,2247.326667,NaN,NaN
10405,ETH-USDT,2023-12-15 14:00:00,2252.01,2268.503191,2266.513913,2257.634167,2250.666667,2247.003333,NaN,1.702655e+09


In [1]:
t7[t7.SL_time.notnull()].shape[0]

NameError: name 't7' is not defined

In [77]:
# Good idea to save your work. Actually more frequently than this.

t7.to_csv("drive/My Drive/t7.csv")

In [7]:
t7=pd.read_csv("drive/My Drive/t7.csv")

In [9]:
t7.head(5)

,Unnamed: 0,pair,date_time,close,MA_8,MA_4,MA_2,MA_1,MA_30_min,TP_time,SL_time
0,0,BTC-USDT,2022-10-25 13:00:00,19335.83,19302.483263,19287.277292,19297.542917,19295.105000,19298.093333,1.666713e+09,NaN
1,1,BTC-USDT,2022-10-25 13:05:00,19348.41,19302.476947,19288.516667,19299.646667,19297.725000,19307.388333,1.666713e+09,NaN
2,2,BTC-USDT,2022-10-25 13:10:00,19390.18,19302.807684,19289.834792,19302.358750,19302.470833,19321.411667,1.666713e+09,NaN
3,3,BTC-USDT,2022-10-25 13:15:00,19400.22,19303.567053,19292.297500,19306.561250,19310.869167,19339.755000,1.666713e+09,NaN
4,4,BTC-USDT,2022-10-25 13:20:00,19393.21,19304.624316,19294.592500,19311.478333,19320.099167,19354.273333,1.666713e+09,NaN


In [10]:
# give a better name
date_for_model = t7.copy()

# Understanding of the following logic - to define Target - is very important.

In [11]:
# Define the Target variable. If SL happend before TP, it means no response. Response = TP happening.
import numpy as np

date_for_model['Y'] = np.where((date_for_model["TP_time"].notnull()) &
                               ((date_for_model["SL_time"].isnull()) |
                                (date_for_model["SL_time"] < date_for_model["TP_time"])),1, 0)



In [12]:
date_for_model.shape[0]

10406

# Practical Note: Use a lot of paranthesis when you write logical conditions. Sequence of operations can be tricky. By using paranthesis, you take control of the sequence.

# We need to check this logic carefully. In general, we need to check datasteps frequntly. Different ways: checking count of observations, and other related queries.

In [15]:
# Check when TP_time = missing. All should be zero.

date_for_model[date_for_model.TP_time.isnull()]["Y"].sum()

0

In [17]:
# TP not null and SL null. All should be 1.

date_for_model[(date_for_model.TP_time.notnull() & date_for_model.SL_time.isnull())][["TP_time", "SL_time", "Y"]]

,TP_time,SL_time,Y
0,1.666713e+09,NaN,1
1,1.666713e+09,NaN,1
2,1.666713e+09,NaN,1
3,1.666713e+09,NaN,1
4,1.666713e+09,NaN,1
...,...,...,...
10378,1.702496e+09,NaN,1
10379,1.702494e+09,NaN,1
10385,1.702575e+09,NaN,1
10386,1.702574e+09,NaN,1


In [18]:
# TP not null and SL null and TP < SL. All should be 1.

date_for_model[(date_for_model.TP_time.notnull() &
                date_for_model.SL_time.notnull()) &
               (date_for_model.TP_time < date_for_model.SL_time)][["TP_time", "SL_time", "Y"]]

# not such observation. probably because TP_time and SL_time can not happen in one day. This is a great result,
# because we don't need to worry about the sequence of TP and SL, which makes the problem conceptually much simpler.

,TP_time,SL_time,Y


In [19]:
# TP not null and SL null and TP < SL. All should be 0.

date_for_model[(date_for_model.TP_time.notnull() &
                date_for_model.SL_time.notnull()) &
               (date_for_model.TP_time > date_for_model.SL_time)][["TP_time", "SL_time", "Y"]]

,TP_time,SL_time,Y
181,1.667924e+09,1.667918e+09,1
182,1.667924e+09,1.667918e+09,1
183,1.667924e+09,1.667918e+09,1
184,1.667924e+09,1.667918e+09,1
185,1.667924e+09,1.667918e+09,1
...,...,...,...
9874,1.698872e+09,1.698846e+09,1
9875,1.698872e+09,1.698846e+09,1
9876,1.698872e+09,1.698845e+09,1
9877,1.698869e+09,1.698850e+09,1


In [ ]:
# sounds like there are cases where both are non missing; in fact about 10% of the time when TP happened, SL also happened.
# and all the times that both happened, SL happened before TP.

# Assignment: verify these results

# Sounds like Target variable is defined correctly. Next, we need to think about Features to define. We will define several features and will let the model to decide on the ones that have predictive power.

In [20]:
date_for_model["8_4_diff"] = date_for_model["MA_8"] - date_for_model["MA_4"]

In [21]:
date_for_model.shape[0]

10406

In [24]:
date_for_model[["MA_8", "MA_4", "8_4_diff"]]

# you can verify with a calculator

,MA_8,MA_4,8_4_diff
0,19302.483263,19287.277292,15.205971
1,19302.476947,19288.516667,13.960281
2,19302.807684,19289.834792,12.972893
3,19303.567053,19292.297500,11.269553
4,19304.624316,19294.592500,10.031816
...,...,...,...
10401,2269.384043,2269.240870,0.143173
10402,2269.149681,2268.461087,0.688594
10403,2268.939255,2267.776957,1.162299
10404,2268.685000,2267.159130,1.525870


In [25]:
date_for_model["4_2_diff"] = date_for_model["MA_4"] - date_for_model["MA_2"]
date_for_model["2_1_diff"] = date_for_model["MA_2"] - date_for_model["MA_1"]
date_for_model["1_30_diff"] = date_for_model["MA_1"] - date_for_model["MA_30_min"]

date_for_model["Close_8_diff"] = date_for_model["close"] - date_for_model["MA_8"]
date_for_model["Close_4_diff"] = date_for_model["close"] - date_for_model["MA_4"]
date_for_model["Close_2_diff"] = date_for_model["close"] - date_for_model["MA_2"]
date_for_model["Close_1_diff"] = date_for_model["close"] - date_for_model["MA_1"]
date_for_model["Close_30_min_diff"] = date_for_model["close"] - date_for_model["MA_30_min"]

In [30]:
# We also normalize the above features by the Close price (i.e. the money we are supposed to pay), making them more like return
# which is what we are predicting.

date_for_model["8_4_diff_normalized"] = date_for_model["8_4_diff"].div(date_for_model['close'], axis=0)
date_for_model["4_2_diff_normalized"] = date_for_model["4_2_diff"].div(date_for_model['close'], axis=0)
date_for_model["2_1_diff_normalized"] = date_for_model["2_1_diff"].div(date_for_model['close'], axis=0)
date_for_model["1_30_diff_normalized"] = date_for_model["1_30_diff"].div(date_for_model['close'], axis=0)

date_for_model["Close_8_diff_normalized"] = date_for_model["Close_8_diff"].div(date_for_model['close'], axis=0)
date_for_model["Close_4_diff_normalized"] = date_for_model["Close_4_diff"].div(date_for_model['close'], axis=0)
date_for_model["Close_2_diff_normalized"] = date_for_model["Close_2_diff"].div(date_for_model['close'], axis=0)
date_for_model["Close_1_diff_normalized"] = date_for_model["Close_1_diff"].div(date_for_model['close'], axis=0)
date_for_model["Close_30_min_diff_normalized"] = date_for_model["Close_30_min_diff"].div(date_for_model['close'], axis=0)

In [32]:
date_for_model.head(2)

,Unnamed: 0,pair,date_time,close,MA_8,MA_4,MA_2,MA_1,MA_30_min,TP_time,...,Close_30_min_diff,8_4_diff_normalized,4_2_diff_normalized,2_1_diff_normalized,1_30_diff_normalized,Close_8_diff_normalized,Close_4_diff_normalized,Close_2_diff_normalized,Close_1_diff_normalized,Close_30_min_diff_normalized
0,0,BTC-USDT,2022-10-25 13:00:00,19335.83,19302.483263,19287.277292,19297.542917,19295.105,19298.093333,1.666713e+09,...,37.736667,0.000786,-0.000531,0.000126,-0.000155,0.001725,0.002511,0.00198,0.002106,0.001952
1,1,BTC-USDT,2022-10-25 13:05:00,19348.41,19302.476947,19288.516667,19299.646667,19297.725,19307.388333,1.666713e+09,...,41.021667,0.000722,-0.000575,0.000099,-0.000499,0.002374,0.003096,0.00252,0.002620,0.002120


# Data is almost ready. Next Steps:

1. Test-Train Split
2. Data Processing
3. Feature Reduction
4. Model Training (Parameter Tuning)

In [37]:
# Test Train Split - We split based on date (month)
# We need to add a column that shows year_month

import datetime as dt

date_for_model['date'] = pd.to_datetime(date_for_model['date_time'])

In [43]:
date_for_model['year_month'] = date_for_model['date'].apply(lambda x: x.year*100+x.month)

In [46]:
date_for_model.groupby(['year_month']).size()

year_month
202210    182
202211    776
202212    794
202301    796
202302    721
202303    783
202304    748
202305    791
202306    765
202307    767
202308    706
202309    726
202310    761
202311    709
202312    381
dtype: int64

In [63]:
# Let's use 202210 to 202301 as test 1, 202302 to 202310 as train, and 202311 to 202312 as test 2.
# Do you think it is a good segmentation?

Train = date_for_model[(date_for_model.year_month > 202301) & (date_for_model.year_month < 202311)]
Test_1 = date_for_model[(date_for_model.year_month < 202302)]
Test_2 = date_for_model[(date_for_model.year_month > 202310)]

In [49]:
# some columsn will not be used as neither X nor Y. We remove them.
Train.columns

Index(['Unnamed: 0', 'pair', 'date_time', 'close', 'MA_8', 'MA_4', 'MA_2',
       'MA_1', 'MA_30_min', 'TP_time', 'SL_time', 'Y', '8_4_diff', '4_2_diff',
       '2_1_diff', '1_30_diff', 'Close_8_diff', 'Close_4_diff', 'Close_2_diff',
       'Close_1_diff', 'Close_30_min_diff', '8_4_diff_normalized',
       '4_2_diff_normalized', '2_1_diff_normalized', '1_30_diff_normalized',
       'Close_8_diff_normalized', 'Close_4_diff_normalized',
       'Close_2_diff_normalized', 'Close_1_diff_normalized',
       'Close_30_min_diff_normalized', 'date', 'year_month'],
      dtype='object')

In [64]:
Train.drop(["Unnamed: 0", 'date_time', 'date', 'year_month'], axis = 1, inplace=True)
Test_1.drop(["Unnamed: 0", 'date_time', 'date', 'year_month'], axis = 1, inplace=True)
Test_2.drop(["Unnamed: 0", 'date_time', 'date', 'year_month'], axis = 1, inplace=True)

<ipython-input-64-be7fa9243e3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Train.drop(["Unnamed: 0", 'date_time', 'date', 'year_month'], axis = 1, inplace=True)
<ipython-input-64-be7fa9243e3c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_1.drop(["Unnamed: 0", 'date_time', 'date', 'year_month'], axis = 1, inplace=True)
<ipython-input-64-be7fa9243e3c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Test_2.drop(["Unnamed: 

In [65]:
Y_train = Train.Y
X_train = Train.drop(["Y"], axis = 1)

Y_test_1 = Test_1.Y
X_test_1 = Test_1.drop(["Y"], axis = 1)

Y_train = Test_2.Y
X_test_2 = Test_2.drop(["Y"], axis = 1)

In [67]:
X_test_1.columns

Index(['pair', 'close', 'MA_8', 'MA_4', 'MA_2', 'MA_1', 'MA_30_min', 'TP_time',
       'SL_time', '8_4_diff', '4_2_diff', '2_1_diff', '1_30_diff',
       'Close_8_diff', 'Close_4_diff', 'Close_2_diff', 'Close_1_diff',
       'Close_30_min_diff', '8_4_diff_normalized', '4_2_diff_normalized',
       '2_1_diff_normalized', '1_30_diff_normalized',
       'Close_8_diff_normalized', 'Close_4_diff_normalized',
       'Close_2_diff_normalized', 'Close_1_diff_normalized',
       'Close_30_min_diff_normalized'],
      dtype='object')

In [70]:
print (X_train.shape[0])
print (X_test_1.shape[0])
print (X_test_2.shape[0])

6768
2548
1090


# Next step: data processing. We only need to one-hot encode the pair column.

In [72]:
set(date_for_model.pair)

{'BTC-USDT', 'ETH-USDT'}

In [75]:
X_train['Pair_Numerical'] = np.where((X_train["pair"] == 'BTC-USDT'),1, 0)
X_train.drop(["pair"], axis = 1, inplace = True)

X_test_1['Pair_Numerical'] = np.where((X_test_1["pair"] == 'BTC-USDT'),1, 0)
X_test_1.drop(["pair"], axis = 1, inplace = True)

X_test_2['Pair_Numerical'] = np.where((X_test_2["pair"] == 'BTC-USDT'),1, 0)
X_test_2.drop(["pair"], axis = 1, inplace = True)

In [76]:
X_train.columns

Index(['close', 'MA_8', 'MA_4', 'MA_2', 'MA_1', 'MA_30_min', 'TP_time',
       'SL_time', '8_4_diff', '4_2_diff', '2_1_diff', '1_30_diff',
       'Close_8_diff', 'Close_4_diff', 'Close_2_diff', 'Close_1_diff',
       'Close_30_min_diff', '8_4_diff_normalized', '4_2_diff_normalized',
       '2_1_diff_normalized', '1_30_diff_normalized',
       'Close_8_diff_normalized', 'Close_4_diff_normalized',
       'Close_2_diff_normalized', 'Close_1_diff_normalized',
       'Close_30_min_diff_normalized', 'Pair_Numerical'],
      dtype='object')

# Data is ready to train the model. No need for feature reduction, as there are not many of them.

# Assignment: Perform a Grid Search on an XGB model and find the optimum set of parameters.